In [1]:
import tensorflow as tf
import numpy as np
import os
LOAD_MODEL = 1
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.__version__

Num GPUs Available:  0


'2.3.1'

In [2]:
def read_data(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-4]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    data = np.array(data).transpose(0, 2, 1)
    d_shape = data.shape
    print(d_shape)
    return data.reshape(d_shape[0], 1, d_shape[1], d_shape[2])

def read_label(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-10]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    return np.array(data)

def save_output_to_file(output):
    output = output.reshape((1024,1))
    for idx, value in enumerate(output):
        if value > 0.5:
            output[idx] = 1
        else:
            output[idx] = 0

    with open('test.txt', 'w+') as file:
        file.write(np.array2string(output))

In [3]:
prefix_path = os.path.abspath(f'../../data/Dataset7')
train_data_path = os.path.abspath(f'{prefix_path}/train/train_data')
train_label_path = f'{prefix_path}/train/train_label'
test_data_path = f'{prefix_path}/test/test_data'
test_label_path = f'{prefix_path}/test/test_label'

train_data = read_data(train_data_path).transpose(0, 2, 3, 1) # 519
train_label = read_label(train_label_path)
test_data = read_data(test_data_path).transpose(0, 2, 3, 1) # 519
test_label = read_label(test_label_path)

print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

(519, 6, 1024)
(58, 6, 1024)
(519, 6, 1024, 1)
(519, 1024)
(58, 6, 1024, 1)
(58, 1024)


In [4]:
# Training Parameters
epochs = 140
batch_size = 32
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

def my_loss_fn(y_true, y_pred):
    return -tf.reduce_mean(input_tensor=y_true*tf.math.log(tf.clip_by_value(y_pred,1e-10,1.0))+(1-y_true)*tf.math.log(tf.clip_by_value(1-y_pred,1e-10,1.0)), name="cross_entropy")

In [5]:
input1 = tf.keras.Input(shape=(6,1024,1))

conv1_1 = tf.keras.layers.Conv2D(64, [1, 16], activation='relu', padding='same')(input1)
conv1_2 = tf.keras.layers.Conv2D(64, [1, 16], activation='relu', padding='same')(conv1_1)
    
conv2_1 = tf.keras.layers.MaxPooling2D(pool_size=[1, 2], strides=[1, 2], padding='same')(conv1_2)
conv2_2 = tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same')(conv2_1)
conv2_3 = tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same')(conv2_2)

conv3_1 = tf.keras.layers.MaxPooling2D(pool_size=[1, 2], strides=[1, 2], padding='same')(conv2_3)
conv3_2 = tf.keras.layers.Conv2D(256, [1, 16], activation='relu', padding='same')(conv3_1)
conv3_3 = tf.keras.layers.Conv2D(256, [1, 16], activation='relu', padding='same')(conv3_2)
conv3_4 = tf.keras.layers.Conv2D(256, [1, 16], activation='relu', padding='same')(conv3_3)
    
conv2_4_1 = tf.keras.layers.Conv2DTranspose(128, [1, 2], strides=[1, 2], padding='same')(conv3_4)
conv2_4 = tf.keras.layers.Concatenate(axis=3)([conv2_4_1, conv2_3])
conv2_5 = tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same')(conv2_4)
conv2_6 = tf.keras.layers.Conv2D(128, [1, 16], activation='relu', padding='same')(conv2_5)
    
conv1_3_1 = tf.keras.layers.Conv2DTranspose(64, [1, 2], strides=[1, 2], padding='same')(conv2_6)
conv1_3 = tf.keras.layers.Concatenate(axis=3)([conv1_2, conv1_3_1])
conv1_4 = tf.keras.layers.Conv2D(64, [1, 16], activation='relu', padding='same')(conv1_3)
conv1_5 = tf.keras.layers.Conv2D(256, [6, 1], activation='relu')(conv1_4)
conv1_6 = tf.keras.layers.Conv2D(1, [1, 1], activation='sigmoid')(conv1_5)

out = tf.keras.layers.Reshape((-1,))(conv1_6)

model = tf.keras.models.Model(inputs=[input1], outputs=[out])


In [6]:
for layer in model.layers:
    print(layer.output_shape)

[(None, 6, 1024, 1)]
(None, 6, 1024, 64)
(None, 6, 1024, 64)
(None, 6, 512, 64)
(None, 6, 512, 128)
(None, 6, 512, 128)
(None, 6, 256, 128)
(None, 6, 256, 256)
(None, 6, 256, 256)
(None, 6, 256, 256)
(None, 6, 512, 128)
(None, 6, 512, 256)
(None, 6, 512, 128)
(None, 6, 512, 128)
(None, 6, 1024, 64)
(None, 6, 1024, 128)
(None, 6, 1024, 64)
(None, 1, 1024, 256)
(None, 1, 1024, 1)
(None, 1024)


In [7]:
model.compile(optimizer=optimizer,
              loss=my_loss_fn,
              metrics=['mae'])

# Saving the model during training

In [8]:
checkpoint_path = "training_1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [9]:
# load model
if LOAD_MODEL == 1:
    model.load_weights(checkpoint_path)

In [10]:
model.fit(x=train_data,
          y=train_label,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_data, test_label),
          callbacks=[cp_callback])

Epoch 1/150
 1/17 [>.............................] - ETA: 0s - loss: 0.3795 - mae: 0.2543

KeyboardInterrupt: 

In [ ]:
# get an output by predicting the data
output = model.predict(train_data[:1])
save_output_to_file(output)